# 時系列データとは何か，その基本的な取り扱いについて調べる(2022/01/25)
---
## 概要
---
データの形式の一つである時系列データについて，その特性・分析手法(トレンド，周期性，移動平均，季節調整，ノイズ，階差，スペクトル，自己相関，相互相関，自己回帰モデル)について調べる．

## 時系列データ
---


## 参考文献

[時系列，Wikipedia](https://ja.wikipedia.org/wiki/%E6%99%82%E7%B3%BB%E5%88%97)

[時系列データの解析，東京大学　数理・情報教育研究センター](http://www.mi.u-tokyo.ac.jp/consortium2/pdf/4-4_literacy_level_note.pdf)

[32-3. 時系列データと移動平均，統計WEB](https://bellcurve.jp/statistics/course/12933.html)

[32-4. 時系列データにおける周期変動，統計WEB](https://bellcurve.jp/statistics/course/23739.html)

[32-5. 自己相関，統計WEB](https://bellcurve.jp/statistics/course/12935.html)

[【テックコラム】時系列データの基礎と可視化，DataCurrent](https://www.datacurrent.co.jp/column/time-series-visualization20210628/)

[時系列相関，株式会社アイスタット](https://istat.co.jp/ji_commentary/time_series)